# Jane Street Real-Time Market Data Forecasting: LSTM from Scratch

This notebook implements an LSTM from scratch using PyTorch for the Jane Street competition.

## Competition Overview

**Goal:** Predict financial market responses for real-time trading decisions

**Challenge:**
- **130 anonymized features** representing market data
- **Multiple responders** (resp_1, resp_2, resp_3, resp_4, resp) - different time horizons
- **Time-ordered data** with date_id and time_id
- **Weight column** - importance of each prediction
- **Real-time inference** constraint

**Key Difficulties:**
1. Concept drift - relationships change over time
2. Low signal-to-noise ratio
3. High-frequency data with volatility
4. Multicollinearity among features
5. Need for fast inference

## Why LSTM?

**Advantages for this problem:**
- Captures temporal dependencies in market data
- Cell state provides long-term memory
- Gates handle regime changes (different market conditions)
- Can learn which features matter over time

## What We'll Build

1. **LSTM Cell from scratch** - understand gating mechanisms
2. **Complete LSTM model** for sequence modeling
3. **Data preprocessing** - feature engineering & normalization
4. **Training pipeline** - with proper validation
5. **Evaluation metrics** - weighted R² score
6. **Inference optimization** - for real-time prediction

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

## Step 1: Generate Synthetic Jane Street-like Data

Since we don't have the actual competition data, we'll create synthetic data that mimics the characteristics:
- 130 features (mix of trend, stationary, and noisy)
- Time-ordered sequences
- Multiple responders
- Weights
- Concept drift

In [ ]:
def generate_jane_street_like_data(n_samples=100000, n_features=130, seq_length=10):
    """
    Generate synthetic data mimicking Jane Street competition.
    
    Args:
        n_samples: Number of time points
        n_features: Number of features (130 in actual competition)
        seq_length: Sequence length for LSTM
        
    Returns:
        DataFrame with features and responders
    """
    print("Generating synthetic Jane Street-like data...")
    
    # Time indices
    time = np.arange(n_samples)
    date_id = time // 500  # ~500 observations per day
    time_id = time % 500
    
    # Feature categories (mimicking actual data patterns)
    features = {}
    
    # Group 1: Features with trend (40 features)
    for i in range(40):
        trend = 0.001 * time * np.random.randn()
        seasonal = 2 * np.sin(2 * np.pi * time / (50 + i * 10))
        noise = np.random.randn(n_samples) * 0.5
        features[f'feature_{i}'] = trend + seasonal + noise
    
    # Group 2: Stationary features (40 features)
    for i in range(40, 80):
        features[f'feature_{i}'] = np.random.randn(n_samples)
    
    # Group 3: Features with heteroscedasticity (30 features)
    for i in range(80, 110):
        volatility = 0.5 + 0.5 * np.abs(np.sin(2 * np.pi * time / 1000))
        features[f'feature_{i}'] = np.random.randn(n_samples) * volatility
    
    # Group 4: Correlated features (20 features)
    base_feature = np.random.randn(n_samples)
    for i in range(110, 130):
        noise = np.random.randn(n_samples) * 0.3
        features[f'feature_{i}'] = base_feature + noise
    
    # Create DataFrame
    df = pd.DataFrame(features)
    df['date_id'] = date_id
    df['time_id'] = time_id
    
    # Generate responders (different time horizons)
    # Response depends on recent feature values with some noise
    feature_matrix = df[[f'feature_{i}' for i in range(130)]].values
    
    # Weights for different features (some more important)
    importance = np.random.exponential(0.1, 130)
    importance = importance / importance.sum()
    
    # resp_1: short-term (next step)
    df['resp_1'] = feature_matrix @ importance + np.random.randn(n_samples) * 0.5
    
    # resp_2: medium-term (rolling average)
    df['resp_2'] = pd.Series(feature_matrix @ importance).rolling(5, min_periods=1).mean() + \
                   np.random.randn(n_samples) * 0.3
    
    # resp_3: longer-term
    df['resp_3'] = pd.Series(feature_matrix @ importance).rolling(10, min_periods=1).mean() + \
                   np.random.randn(n_samples) * 0.2
    
    # resp_4: longest-term
    df['resp_4'] = pd.Series(feature_matrix @ importance).rolling(20, min_periods=1).mean() + \
                   np.random.randn(n_samples) * 0.15
    
    # Main responder (combination)
    df['resp'] = (df['resp_1'] + df['resp_2'] + df['resp_3'] + df['resp_4']) / 4
    
    # Weight (importance of prediction)
    # Lower weight for more volatile predictions
    volatility_proxy = df[['feature_0', 'feature_1', 'feature_2']].std(axis=1)
    df['weight'] = 1.0 / (1.0 + volatility_proxy)
    df['weight'] = df['weight'] / df['weight'].mean()  # Normalize
    
    # About 17% of samples have weight = 0 (as in actual competition)
    zero_weight_mask = np.random.rand(n_samples) < 0.17
    df.loc[zero_weight_mask, 'weight'] = 0
    
    print(f"Generated data shape: {df.shape}")
    print(f"Features: {n_features}")
    print(f"Responders: 5 (resp_1, resp_2, resp_3, resp_4, resp)")
    print(f"Zero weight samples: {(df['weight'] == 0).sum()} ({(df['weight'] == 0).mean()*100:.1f}%)")
    
    return df


# Generate data
n_samples = 100000
n_features = 130
df = generate_jane_street_like_data(n_samples, n_features)

print("\nData statistics:")
print(df[['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4', 'weight']].describe())

In [ ]:
# Visualize data characteristics
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Plot 1: Sample features over time
sample_features = ['feature_0', 'feature_50', 'feature_100']
for feat in sample_features:
    axes[0, 0].plot(df[feat][:2000], alpha=0.7, label=feat)
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Value')
axes[0, 0].set_title('Sample Features Over Time', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Responders
responders = ['resp_1', 'resp_2', 'resp_3', 'resp_4']
for resp in responders:
    axes[0, 1].plot(df[resp][:2000], alpha=0.7, label=resp)
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Response')
axes[0, 1].set_title('Responders (Different Time Horizons)', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Weight distribution
axes[0, 2].hist(df[df['weight'] > 0]['weight'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 2].set_xlabel('Weight')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('Weight Distribution (non-zero)', fontweight='bold')
axes[0, 2].grid(True, alpha=0.3)

# Plot 4: Feature correlation heatmap (subset)
feature_subset = [f'feature_{i}' for i in [0, 10, 20, 30, 40, 50]]
corr = df[feature_subset].corr()
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', ax=axes[1, 0], 
            cbar_kws={'label': 'Correlation'})
axes[1, 0].set_title('Feature Correlation (Subset)', fontweight='bold')

# Plot 5: Resp vs weight scatter
sample_idx = np.random.choice(len(df), 5000, replace=False)
axes[1, 1].scatter(df.loc[sample_idx, 'resp'], 
                   df.loc[sample_idx, 'weight'], 
                   alpha=0.3, s=10)
axes[1, 1].set_xlabel('Response')
axes[1, 1].set_ylabel('Weight')
axes[1, 1].set_title('Response vs Weight', fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

# Plot 6: Daily response distribution
daily_resp = df.groupby('date_id')['resp'].mean()
axes[1, 2].plot(daily_resp.values)
axes[1, 2].set_xlabel('Date ID')
axes[1, 2].set_ylabel('Mean Response')
axes[1, 2].set_title('Daily Mean Response (Trend)', fontweight='bold')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Step 2: LSTM Cell from Scratch

### LSTM Mathematics

LSTM has **4 gates** that control information flow:

**1. Forget Gate** (what to remove from cell state):
```
f_t = σ(W_f @ [h_{t-1}, x_t] + b_f)
```

**2. Input Gate** (what new information to add):
```
i_t = σ(W_i @ [h_{t-1}, x_t] + b_i)
g_t = tanh(W_g @ [h_{t-1}, x_t] + b_g)  # Candidate values
```

**3. Cell State Update** (combine forget and input):
```
c_t = f_t ⊙ c_{t-1} + i_t ⊙ g_t
```

**4. Output Gate** (what to output):
```
o_t = σ(W_o @ [h_{t-1}, x_t] + b_o)
h_t = o_t ⊙ tanh(c_t)
```

Where:
- σ = sigmoid (0 to 1, acts as gate)
- tanh = hyperbolic tangent (-1 to 1)
- ⊙ = element-wise multiplication

In [ ]:
class LSTMCell(nn.Module):
    """
    LSTM Cell implemented from scratch.
    
    Key insight: Gates (forget, input, output) control information flow,
    preventing vanishing gradients and enabling long-term dependencies.
    """
    
    def __init__(self, input_size, hidden_size):
        """
        Args:
            input_size: Dimension of input features
            hidden_size: Dimension of hidden state and cell state
        """
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Combined weight matrix for all gates
        # Concatenate [h_{t-1}, x_t], so input dimension is hidden_size + input_size
        combined_size = hidden_size + input_size
        
        # Forget gate
        self.W_f = nn.Linear(combined_size, hidden_size, bias=True)
        
        # Input gate
        self.W_i = nn.Linear(combined_size, hidden_size, bias=True)
        
        # Candidate cell state (sometimes called g)
        self.W_g = nn.Linear(combined_size, hidden_size, bias=True)
        
        # Output gate
        self.W_o = nn.Linear(combined_size, hidden_size, bias=True)
        
        self._init_weights()
    
    def _init_weights(self):
        """Xavier initialization for stable gradients"""
        for weight in [self.W_f, self.W_i, self.W_g, self.W_o]:
            nn.init.xavier_uniform_(weight.weight)
            nn.init.zeros_(weight.bias)
    
    def forward(self, x_t, h_prev, c_prev):
        """
        Forward pass for one time step.
        
        Args:
            x_t: Input at time t [batch, input_size]
            h_prev: Previous hidden state [batch, hidden_size]
            c_prev: Previous cell state [batch, hidden_size]
            
        Returns:
            h_t: New hidden state [batch, hidden_size]
            c_t: New cell state [batch, hidden_size]
        """
        # Concatenate previous hidden state and current input
        combined = torch.cat([h_prev, x_t], dim=1)  # [batch, hidden_size + input_size]
        
        # === GATE COMPUTATIONS ===
        
        # Forget gate: what to forget from previous cell state
        f_t = torch.sigmoid(self.W_f(combined))
        
        # Input gate: what new information to add
        i_t = torch.sigmoid(self.W_i(combined))
        
        # Candidate cell state: new information to potentially add
        g_t = torch.tanh(self.W_g(combined))
        
        # Output gate: what to output from cell state
        o_t = torch.sigmoid(self.W_o(combined))
        
        # === STATE UPDATES ===
        
        # Update cell state: forget old + add new
        c_t = f_t * c_prev + i_t * g_t
        
        # Update hidden state: output gate applied to cell state
        h_t = o_t * torch.tanh(c_t)
        
        return h_t, c_t


print("✓ LSTM Cell implemented!")
print("\nLSTM has 4 components:")
print("  1. Forget gate - what to remove from memory")
print("  2. Input gate - what new info to store")
print("  3. Cell state - long-term memory")
print("  4. Output gate - what to expose as hidden state")

## Step 3: Complete LSTM Model for Jane Street

**Architecture:**
```
Input: [batch, seq_len, n_features]
  ↓
LSTM Layers (2 layers, 128 hidden units)
  ↓
Dropout (0.3)
  ↓
Fully Connected (128 → 64)
  ↓
Dropout (0.2)
  ↓
Output (64 → 5)  [resp_1, resp_2, resp_3, resp_4, resp]
```

In [ ]:
class LSTM_JaneStreet(nn.Module):
    """
    Complete LSTM model for Jane Street competition.
    Built from scratch using our custom LSTM cells.
    """
    
    def __init__(self, input_size, hidden_size=128, num_layers=2, 
                 output_size=5, dropout=0.3):
        """
        Args:
            input_size: Number of features (130 for Jane Street)
            hidden_size: Hidden state dimension
            num_layers: Number of stacked LSTM layers
            output_size: Number of responders to predict (5)
            dropout: Dropout rate for regularization
        """
        super(LSTM_JaneStreet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        
        # Stack of LSTM cells
        self.lstm_cells = nn.ModuleList()
        for i in range(num_layers):
            cell_input_size = input_size if i == 0 else hidden_size
            self.lstm_cells.append(LSTMCell(cell_input_size, hidden_size))
        
        # Dropout for regularization
        self.dropout = nn.Dropout(dropout)
        
        # Output layers
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, output_size)
        
        # Batch normalization (helps with training stability)
        self.bn = nn.BatchNorm1d(64)
    
    def forward(self, x, hidden=None):
        """
        Forward pass through LSTM.
        
        Args:
            x: Input sequences [batch, seq_len, input_size]
            hidden: Initial (h, c) states (optional)
            
        Returns:
            output: Predictions [batch, output_size]
            (h_n, c_n): Final hidden and cell states
        """
        batch_size = x.size(0)
        seq_len = x.size(1)
        
        # Initialize hidden and cell states if not provided
        if hidden is None:
            h, c = self._init_hidden(batch_size, x.device)
        else:
            h, c = hidden
        
        # Process sequence one time step at a time
        for t in range(seq_len):
            x_t = x[:, t, :]  # [batch, input_size]
            
            # Pass through each LSTM layer
            for layer in range(self.num_layers):
                h[layer], c[layer] = self.lstm_cells[layer](x_t, h[layer], c[layer])
                x_t = h[layer]  # Output of this layer → input to next
                
                # Apply dropout between layers (not on last layer)
                if layer < self.num_layers - 1:
                    x_t = self.dropout(x_t)
        
        # Use final hidden state for prediction
        final_hidden = h[-1]  # [batch, hidden_size]
        
        # Output layers
        out = torch.relu(self.fc1(final_hidden))
        out = self.bn(out)
        out = self.dropout(out)
        out = self.fc2(out)  # [batch, output_size]
        
        return out, (h, c)
    
    def _init_hidden(self, batch_size, device):
        """Initialize hidden and cell states to zeros"""
        h = [torch.zeros(batch_size, self.hidden_size, device=device)
             for _ in range(self.num_layers)]
        c = [torch.zeros(batch_size, self.hidden_size, device=device)
             for _ in range(self.num_layers)]
        return h, c


# Create model
model = LSTM_JaneStreet(
    input_size=n_features,
    hidden_size=128,
    num_layers=2,
    output_size=5,  # Predict all 5 responders
    dropout=0.3
).to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Model Architecture:")
print("=" * 70)
print(model)
print("=" * 70)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

## Step 4: Data Preprocessing and Dataset Creation

In [ ]:
class JaneStreetDataset(Dataset):
    """
    Dataset for Jane Street time series.
    Creates sequences of features → multiple responders.
    """
    
    def __init__(self, df, seq_length=10, target_cols=None):
        """
        Args:
            df: DataFrame with features and responders
            seq_length: Length of input sequences
            target_cols: List of responder column names
        """
        self.seq_length = seq_length
        
        if target_cols is None:
            target_cols = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']
        
        # Extract features
        feature_cols = [col for col in df.columns if col.startswith('feature_')]
        self.features = df[feature_cols].values.astype(np.float32)
        
        # Extract targets
        self.targets = df[target_cols].values.astype(np.float32)
        
        # Extract weights
        self.weights = df['weight'].values.astype(np.float32)
        
        # Normalize features (important!)
        self.scaler = StandardScaler()
        self.features = self.scaler.fit_transform(self.features)
        
        # Normalize targets
        self.target_mean = self.targets.mean(axis=0)
        self.target_std = self.targets.std(axis=0)
        self.targets = (self.targets - self.target_mean) / (self.target_std + 1e-8)
        
        print(f"Dataset created: {len(self)} samples")
        print(f"Feature shape: {self.features.shape}")
        print(f"Target shape: {self.targets.shape}")
    
    def __len__(self):
        return len(self.features) - self.seq_length
    
    def __getitem__(self, idx):
        # Get sequence of features
        x = self.features[idx : idx + self.seq_length]  # [seq_length, n_features]
        
        # Get target (next time step after sequence)
        y = self.targets[idx + self.seq_length]  # [n_responders]
        
        # Get weight
        w = self.weights[idx + self.seq_length]
        
        return (
            torch.FloatTensor(x),
            torch.FloatTensor(y),
            torch.FloatTensor([w])
        )


# Create datasets
seq_length = 10

# Split data: 70% train, 15% val, 15% test (time-ordered)
train_size = int(0.7 * len(df))
val_size = int(0.15 * len(df))

train_df = df[:train_size].reset_index(drop=True)
val_df = df[train_size:train_size + val_size].reset_index(drop=True)
test_df = df[train_size + val_size:].reset_index(drop=True)

train_dataset = JaneStreetDataset(train_df, seq_length)
val_dataset = JaneStreetDataset(val_df, seq_length)
test_dataset = JaneStreetDataset(test_df, seq_length)

# Create dataloaders
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"\nBatch size: {batch_size}")
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

## Step 5: Training Setup

**Loss Function:** Weighted MSE
- Different weights for different predictions
- Zero-weight samples don't contribute to loss

**Metric:** Weighted R² Score (competition metric)

In [ ]:
def weighted_mse_loss(predictions, targets, weights):
    """
    Weighted Mean Squared Error.
    Samples with higher weight contribute more to loss.
    """
    squared_errors = (predictions - targets) ** 2
    weighted_errors = squared_errors * weights.unsqueeze(1)
    return weighted_errors.sum() / (weights.sum() * targets.size(1) + 1e-8)


def weighted_r2_score(predictions, targets, weights):
    """
    Weighted R² score (competition metric).
    R² = 1 - (SS_res / SS_tot)
    """
    # Remove zero-weight samples
    mask = weights.squeeze() > 0
    if mask.sum() == 0:
        return 0.0
    
    pred = predictions[mask]
    true = targets[mask]
    w = weights[mask]
    
    # Weighted mean
    weighted_mean = (true * w).sum(dim=0) / w.sum()
    
    # Sum of squared residuals (weighted)
    ss_res = ((true - pred) ** 2 * w).sum()
    
    # Total sum of squares (weighted)
    ss_tot = ((true - weighted_mean) ** 2 * w).sum()
    
    r2 = 1 - (ss_res / (ss_tot + 1e-8))
    return r2.item()


# Optimizer and scheduler
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3, verbose=True
)

print("Training setup:")
print(f"  Loss: Weighted MSE")
print(f"  Metric: Weighted R² Score")
print(f"  Optimizer: AdamW (lr=0.001, weight_decay=1e-5)")
print(f"  Scheduler: ReduceLROnPlateau")

## Step 6: Training Loop

In [ ]:
def train_epoch(model, dataloader, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    all_preds = []
    all_targets = []
    all_weights = []
    
    for x, y, w in tqdm(dataloader, desc='Training', leave=False):
        x, y, w = x.to(device), y.to(device), w.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        predictions, _ = model(x)
        
        # Compute weighted loss
        loss = weighted_mse_loss(predictions, y, w)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
        all_preds.append(predictions.detach())
        all_targets.append(y)
        all_weights.append(w)
    
    # Calculate metrics
    avg_loss = total_loss / len(dataloader)
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    all_weights = torch.cat(all_weights)
    r2 = weighted_r2_score(all_preds, all_targets, all_weights)
    
    return avg_loss, r2


def evaluate(model, dataloader, device):
    """Evaluate model"""
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    all_weights = []
    
    with torch.no_grad():
        for x, y, w in dataloader:
            x, y, w = x.to(device), y.to(device), w.to(device)
            
            predictions, _ = model(x)
            loss = weighted_mse_loss(predictions, y, w)
            
            total_loss += loss.item()
            all_preds.append(predictions)
            all_targets.append(y)
            all_weights.append(w)
    
    avg_loss = total_loss / len(dataloader)
    all_preds = torch.cat(all_preds)
    all_targets = torch.cat(all_targets)
    all_weights = torch.cat(all_weights)
    r2 = weighted_r2_score(all_preds, all_targets, all_weights)
    
    return avg_loss, r2


# Training
n_epochs = 30
train_losses, train_r2s = [], []
val_losses, val_r2s = [], []
best_val_r2 = -float('inf')

print("Starting training...")
print("=" * 70)

for epoch in range(n_epochs):
    # Train
    train_loss, train_r2 = train_epoch(model, train_loader, optimizer, device)
    train_losses.append(train_loss)
    train_r2s.append(train_r2)
    
    # Validate
    val_loss, val_r2 = evaluate(model, val_loader, device)
    val_losses.append(val_loss)
    val_r2s.append(val_r2)
    
    # Update learning rate
    scheduler.step(val_r2)
    
    # Save best model
    if val_r2 > best_val_r2:
        best_val_r2 = val_r2
        torch.save(model.state_dict(), 'best_jane_street_lstm.pth')
        best_epoch = epoch
    
    # Print progress
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch [{epoch+1:2d}/{n_epochs}] "
          f"Train Loss: {train_loss:.6f} R²: {train_r2:.4f} | "
          f"Val Loss: {val_loss:.6f} R²: {val_r2:.4f} | "
          f"LR: {current_lr:.6f}"
          f"{' ← BEST' if val_r2 == best_val_r2 else ''}")

print("=" * 70)
print(f"Training completed!")
print(f"Best validation R²: {best_val_r2:.4f} (epoch {best_epoch + 1})")

## Step 7: Visualize Training Progress

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Loss curves
axes[0].plot(train_losses, label='Train Loss', linewidth=2)
axes[0].plot(val_losses, label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Weighted MSE Loss', fontsize=12)
axes[0].set_title('Training Progress - Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# Plot 2: R² scores
axes[1].plot(train_r2s, label='Train R²', linewidth=2)
axes[1].plot(val_r2s, label='Val R²', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Weighted R² Score', fontsize=12)
axes[1].set_title('Training Progress - R² Score', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## Step 8: Test Set Evaluation

In [ ]:
# Load best model
model.load_state_dict(torch.load('best_jane_street_lstm.pth'))
model.eval()

# Evaluate on test set
test_loss, test_r2 = evaluate(model, test_loader, device)

print("Test Set Performance:")
print("=" * 50)
print(f"Test Loss (Weighted MSE): {test_loss:.6f}")
print(f"Test R² Score: {test_r2:.4f}")
print("=" * 50)

# Get predictions for visualization
all_predictions = []
all_targets = []
all_weights = []

with torch.no_grad():
    for x, y, w in test_loader:
        x = x.to(device)
        pred, _ = model(x)
        all_predictions.append(pred.cpu().numpy())
        all_targets.append(y.numpy())
        all_weights.append(w.numpy())

predictions = np.concatenate(all_predictions)
targets = np.concatenate(all_targets)
weights = np.concatenate(all_weights)

# Denormalize predictions
predictions_denorm = predictions * test_dataset.target_std + test_dataset.target_mean
targets_denorm = targets * test_dataset.target_std + test_dataset.target_mean

print(f"\nPredictions shape: {predictions_denorm.shape}")
print(f"Targets shape: {targets_denorm.shape}")

In [ ]:
# Visualize predictions vs actual
responder_names = ['resp_1', 'resp_2', 'resp_3', 'resp_4', 'resp']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.ravel()

for i, resp_name in enumerate(responder_names):
    # Scatter plot: predicted vs actual
    mask = weights.squeeze() > 0  # Only non-zero weights
    sample_size = min(2000, mask.sum())
    sample_idx = np.random.choice(np.where(mask)[0], sample_size, replace=False)
    
    axes[i].scatter(targets_denorm[sample_idx, i], 
                   predictions_denorm[sample_idx, i],
                   alpha=0.3, s=10)
    
    # Perfect prediction line
    min_val = min(targets_denorm[sample_idx, i].min(), 
                  predictions_denorm[sample_idx, i].min())
    max_val = max(targets_denorm[sample_idx, i].max(), 
                  predictions_denorm[sample_idx, i].max())
    axes[i].plot([min_val, max_val], [min_val, max_val], 
                'r--', linewidth=2, label='Perfect Prediction')
    
    axes[i].set_xlabel('Actual', fontsize=11)
    axes[i].set_ylabel('Predicted', fontsize=11)
    axes[i].set_title(f'{resp_name}', fontsize=12, fontweight='bold')
    axes[i].legend(fontsize=9)
    axes[i].grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])

plt.suptitle('Predictions vs Actual (Test Set)', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## Step 9: Feature Importance Analysis

Analyze which features the LSTM is focusing on.

In [ ]:
def compute_feature_importance(model, dataloader, device):
    """
    Compute feature importance using gradient-based method.
    """
    model.eval()
    feature_grads = []
    
    for x, y, w in dataloader:
        x = x.to(device)
        x.requires_grad = True
        
        # Forward pass
        pred, _ = model(x)
        
        # Compute gradient of output with respect to input
        pred.sum().backward()
        
        # Store gradients
        feature_grads.append(x.grad.abs().mean(dim=(0, 1)).cpu().numpy())
        
        if len(feature_grads) >= 10:  # Sample 10 batches
            break
    
    # Average across batches
    importance = np.mean(feature_grads, axis=0)
    return importance


# Compute importance
print("Computing feature importance...")
importance = compute_feature_importance(model, test_loader, device)

# Plot top 20 important features
top_k = 20
top_indices = np.argsort(importance)[-top_k:][::-1]

plt.figure(figsize=(12, 6))
plt.bar(range(top_k), importance[top_indices])
plt.xlabel('Feature Index', fontsize=12)
plt.ylabel('Importance (Gradient Magnitude)', fontsize=12)
plt.title(f'Top {top_k} Most Important Features', fontsize=14, fontweight='bold')
plt.xticks(range(top_k), [f'F{i}' for i in top_indices], rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print(f"\nTop 10 most important features:")
for i, idx in enumerate(top_indices[:10]):
    print(f"  {i+1}. feature_{idx}: {importance[idx]:.6f}")

## Step 10: Time Series Prediction Visualization

In [ ]:
# Visualize predictions over time
n_plot = 500

fig, axes = plt.subplots(3, 2, figsize=(16, 12))
axes = axes.ravel()

for i, resp_name in enumerate(responder_names):
    axes[i].plot(targets_denorm[:n_plot, i], label='Actual', linewidth=2, alpha=0.7)
    axes[i].plot(predictions_denorm[:n_plot, i], label='Predicted', 
                linewidth=2, alpha=0.7, linestyle='--')
    axes[i].set_xlabel('Time Step', fontsize=11)
    axes[i].set_ylabel('Value', fontsize=11)
    axes[i].set_title(f'{resp_name} Over Time', fontsize=12, fontweight='bold')
    axes[i].legend(fontsize=10)
    axes[i].grid(True, alpha=0.3)

# Remove extra subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.show()

## Summary: Key Insights

### 1. LSTM Architecture for Finance

**Why LSTM works for this problem:**
- **Gating mechanism** handles concept drift (changing market regimes)
- **Cell state** maintains long-term dependencies across time
- **Input/forget gates** learn which features matter when
- **Output gate** controls what information to expose

### 2. Implementation Details

**From-scratch components:**
- Custom LSTM cell with all 4 gates
- Manual time step iteration
- Explicit state management (h_t, c_t)

**Key techniques:**
- Batch normalization (training stability)
- Dropout (prevent overfitting)
- Gradient clipping (handle exploding gradients)
- Feature normalization (StandardScaler)

### 3. Jane Street Specific Challenges

**Data characteristics:**
- 130 anonymized features with different patterns
- Multiple time horizons (resp_1 to resp_4)
- Weighted samples (importance varies)
- High noise-to-signal ratio

**Solutions implemented:**
- Weighted loss function
- Sequence modeling (capture temporal patterns)
- Multi-output prediction (all responders)
- Proper time-series split (no data leakage)

### 4. Performance Metrics

**Weighted R² Score:**
- Competition metric
- Accounts for sample importance
- R² > 0: Better than mean prediction
- R² close to 1: Perfect prediction

**Expected results on real data:**
- R² ~ 0.001 - 0.005 is competitive
- Finance data is extremely noisy
- Small improvements = big money

### 5. LSTM vs Other Approaches

**LSTM advantages:**
- Captures temporal dependencies
- Handles variable-length sequences
- Robust to different market regimes

**Alternatives:**
- GRU: Simpler, faster, similar performance
- Transformer: Better for very long sequences
- Tree models (XGBoost): Often strong baseline
- Ensemble: Combine multiple approaches

### 6. Real Competition Tips

**Feature engineering:**
- Rolling statistics (mean, std, min, max)
- Lag features (previous values)
- Technical indicators (momentum, volatility)
- Cross-feature interactions

**Model improvements:**
- Attention mechanism (focus on important time steps)
- Multi-task learning (auxiliary tasks)
- Adversarial validation (detect distribution shift)
- Pseudo-labeling (use test set)

**Validation strategy:**
- Time-series cross-validation
- Out-of-time validation
- Walk-forward analysis
- Monitor stability across folds

### 7. Production Considerations

**Real-time inference:**
- Fast forward pass (<10ms)
- Batch predictions when possible
- Cache hidden states
- Model quantization (FP16)

**Monitoring:**
- Track prediction distribution
- Detect concept drift
- Retrain regularly
- A/B testing

---

**Congratulations! You've built an LSTM from scratch for financial forecasting! 🎉**

This implementation demonstrates:
- Complete LSTM cell mathematics
- Proper time-series modeling
- Financial data handling
- Competition-grade evaluation
- Production-ready considerations